In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('urban-data.csv')
data.dropna()
data.head()

,<empty_1>,Word,Example,Class
0,1,party,n. a social atmosphere that includes mixin' an...,1
1,2,bud,marajuana,1
2,3,junk,the stuff that you leave lying around on your ...,1
3,4,faded,piss drunk!,1
4,5,BYOC,Acronym: Bring Your Own Crap. Similar to [BYOB...,1


Insert simple label function 1

In [3]:
def LB1(x):
    return any(ele in str(x) for ele in ['marajuana', 'Fentanyl', 'heroin', 'weed', 'ganja','drug', 'marijuana'])
LB1(data['Example'][0])
data['LB1']=list(data['Example'].apply(LB1))
data.sort_values(['LB1'])

,<empty_1>,Word,Example,Class,LB1
0,1,party,n. a social atmosphere that includes mixin' an...,1,False
1979,1980,pipe,"1: rope, cool, tight, bomb, rad, awesome. ;; 2...",2,False
1980,1981,cinemuck,the collection of goopy stuff on the movie the...,2,False
1981,1982,tart cart,bus used to bring mentally challenged kids to ...,2,False
1982,1983,linux,"An OS that rarely crashes, runs on most hardwa...",2,False
...,...,...,...,...,...
2826,2827,asshole,Slang: An empty paper towel roll that has been...,2,True
2361,2362,X,n. slang for the drug Ecstasy (C11H15NO2) ;; (...,2,True
2362,2363,xtc,n. slang for the drug Ecstasy (C11H15NO2) ;; (...,2,True
665,666,light-weight,one who can only handle small amounts of vario...,2,True


Insert simple label function 2 (DEA corpus)

In [8]:
r1 = pd.read_csv('DEA drugs term data.CSV')
def LB2(y):
    return any(ele in str(y) for ele in r1['Slang/Code Word'])
data['LB2']=list(data['Example'].apply(LB2))
data.sort_values(['LB2'])

,<empty_1>,Word,Example,Class,LB1,LB2
0,1,party,n. a social atmosphere that includes mixin' an...,1,False,False
1909,1910,assclown,"1. A person who, while under the influence, ma...",2,False,False
1910,1911,cnut,A corruption of a slang term for a lady's part...,2,False,False
1911,1912,fwah,"shit, poop, caca, #2, etc..",2,False,False
1912,1913,fuck a duck,oh shit,2,False,False
...,...,...,...,...,...,...
1660,1661,eye candy,Women that are HOT and you can't take you eyes...,2,False,True
2268,2269,Mangoat,"A Half-man, Half-goat hybrid.",2,False,True
936,937,FWB,'Friends With Benefits' ;; When two people are...,2,False,True
933,934,dilligaf,Do I Look Like I Give A F**k?,2,False,True


Insert simple label function 3 (Urban Dictionary classifier)

In [14]:
r2 = pd.read_csv('urban r2.csv')
def LB3(z):
    return any(ele in str(z) for ele in r2['Word'])
data['LB3']=list(data['Example'].apply(LB3))
data.sort_values(['LB3'])

,<empty_1>,Word,Example,Class,LB1,LB2,LB3
1499,1500,douching swish,A made basket in basketball that not only swis...,2,False,False,False
1560,1561,nimrod,"Orginally from the Biblical Nimrod, a mighty h...",2,False,False,False
1559,1560,snatch,see [cunt],2,False,False,False
1558,1559,net,elite computer hacker,2,False,False,False
1556,1557,sturdidity,The property of being sturdy.,2,False,False,False
...,...,...,...,...,...,...,...
1557,1558,yarmer,"underground slang for a marijuana cigarette, o...",2,True,False,True
1553,1554,sit on the grenade,1. After the man in the black pajamas throws a...,2,False,False,True
1552,1553,rusty trumpet,1. the female version of the [rusty trombone] ...,2,False,False,True
1574,1575,dell holiday n00b,an inexperienced piece of shit who buys a dell...,2,False,False,True


In [ ]:
N-gram feature extraction

In [26]:
import nltk, re, string, collections
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.util import ngrams
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()
def preproces_text(slang):
    # Remove numbers.
    slang = re.sub('\d+', ' ', str(slang))
    
    # Replace newlines with spaces.
    slang = re.sub('\n', ' ', slang)
    
    # Replace punctuation with spaces.
    slang = re.sub(r'[^\w\s]', ' ', slang)
    
    # Replace links.
    slang = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                   'link', str(slang), flags=re.MULTILINE)

    # Replace emails.
    slang = re.sub('\S+@\S+', 'email', slang, flags=re.MULTILINE)
    
    # Convert all letters to lower case.
    slang = slang.lower()
    
    # Tokenizing text.
    words = ''
    
    # Create the stemmer.
    stemmer = SnowballStemmer("english")
    
    # Split text into words.
    slang = slang.split()
    
    for word in slang:
        # Remove stopwords.
        if word not in set(stopwords.words('english')):
            # Stem word.
            words = words + stemmer.stem(word) + " "
    return words

data['Example'] = data['Example'].progress_apply(preproces_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ngohi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ngohi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 2999/2999 [00:13<00:00, 220.09it/s]


Apply Snorkel

In [4]:
from snorkel import labeling
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
lfs=[LB1, LB2, LB3]

ImportError: cannot import name 'labeling' from 'snorkel' (C:\Users\ngohi\anaconda3\lib\site-packages\snorkel\__init__.py)